## Download data

In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm 
import tarfile

cifar10_dataset_folder_path = 'cifar-10-batches-py'

class DownloadProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

""" 
    check if the data (zip) file is already downloaded
    if not, download it from "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz" and save as cifar-10-python.tar.gz
"""
if not isfile('cifar-10-python.tar.gz'):
    with DownloadProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            'cifar-10-python.tar.gz',
            pbar.hook)

if not isdir(cifar10_dataset_folder_path):
    with tarfile.open('cifar-10-python.tar.gz') as tar:
        tar.extractall()
        tar.close()

CIFAR-10 Dataset: 171MB [10:11, 279kB/s]                                                                               


## Exploring the dataset

In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def load_labels():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [4]:
def load_cfar10_batch(dataset_path, batch_id):
    with open(dataset_path + '/data_batch_' + str(batch_id), mode='rb') as f:
        batch = pickle.load(f, encoding='latin1')
        
    features = batch['data']
    labels = batch['labels']
    
    '''print("<<<<<<<<<<<<<<<<<< Before Reshaping >>>>>>>>>>>>>>>>>>>")
    print(features.shape)
    print(np.array(labels).shape)'''
    
    features = features.reshape((len(features), 3, 32, 32)).transpose(0,2,3,1)
    
    '''print("<<<<<<<<<<<<<<<<<< After Reshaping >>>>>>>>>>>>>>>>>>>")
    print(features.shape)'''
    
    return features, labels    

In [5]:
def display_stats(dataset_path, batch_id, sample_id):
    features, labels = load_cfar10_batch(dataset_path, batch_id)
    
    if not (0 <= sample_id < len(features)):
        print('{} samples in batch{}. {} is out of range.'.format(len(features), batch_id, sample_id))
        return None
    
    print('\nStats of batch #{}:'.format(batch_id))
    print('# of Samples: {}\n'.format(len(features)))
    
    label_names = load_label_names()
    label_counts = dict(zip(*np.unique(labels, return_counts=True)))
    for key, value in label_counts.items():
        print('Label Counts of [{}]({}) : {}'.format(key, label_names[key].upper(), value))
    
    sample_image = features[sample_id]
    sample_label = labels[sample_id]
    
    print('\nExample of Image {}:'.format(sample_id))
    print('Image - Min Value: {} Max Value: {}'.format(sample_image.min(), sample_image.max()))
    print('Image - Shape: {}'.format(sample_image.shape))
    print('Label - Label Id: {} Name: {}'.format(sample_label, label_names[sample_label]))
    
    plt.imshow(sample_image)

## Preprocessing

In [6]:
def normalize(x):
    
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x - min_val)/(max_val - min_val)
    return x

In [7]:
def one_hot_encode(x):
    
    encoded = np.zeros((len(x),10))
    
    for idx, val in enumerate(x):
        encoded[idx][val] = 1
        
    return encoded

In [8]:
def _preprocess(features, labels, filename):
    features = normalize(features)
    labels = one_hot_encode(labels)
    
    pickle.dump((features,labels), open(filename, 'wb') )

In [9]:
def preprocess_data(path):
    n_batches = 5
    valid_features = []
    valid_labels = []
    
    print("<<<< Status >>>")
    
    for batch_i in range(1, n_batches + 1):
        features, labels = load_cfar10_batch(path, batch_i)
        
        index_of_validation = int(len(features) * 0.1)
        
        _preprocess(features[:-index_of_validation], labels[:-index_of_validation], 'preprocess_batch_'+str(batch_i) + '.pkl')
        
        valid_features.extend(features[-index_of_validation:])
        valid_labels.extend(labels[-index_of_validation:])
        print('Batch {}: complete!'.format(batch_i))
        
    _preprocess(np.array(valid_features), np.array(valid_labels), 'preprocess_validation.pkl')
    print('Validation batch complete!')
    
    with open(path + '/test_batch', mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')
        
    test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0,2,3,1)
    test_labels = batch['labels']
    
    _preprocess(np.array(test_features), np.array(test_labels), 'preprocess_testing.pkl')
    print('Test batch complete!')

In [10]:
preprocess_data(cifar10_dataset_folder_path)

<<<< Status >>>
Batch 1: complete!
Batch 2: complete!
Batch 3: complete!
Batch 4: complete!
Batch 5: complete!
Validation batch complete!
Test batch complete!


In [11]:
import pickle

valid_features, valid_labels = pickle.load(open('preprocess_validation.pkl', mode='rb'))

## Tensorflow

In [13]:
import tensorflow as tf
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name = 'input_x')
y = tf.placeholder(tf.float32, shape=(None, 10), name = 'output_y')
keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')

In [26]:
def convnet(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape = [3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape = [3, 3, 64, 128], mean=0, stddev=0.08))
    conv3_filter = tf.Variable(tf.truncated_normal(shape = [5, 5, 128, 256], mean=0, stddev=0.08))
    conv4_filter = tf.Variable(tf.truncated_normal(shape = [5, 5, 256, 512], mean=0, stddev=0.08))
    
    #Layers 1,2:
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)
    
    #Layers 3,4:
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    #Layers 5,6:
    conv3 = tf.nn.conv2d(conv2_bn, conv3_filter, strides=[1,1,1,1], padding='SAME')
    conv3 = tf.nn.relu(conv3)
    conv3_pool = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv3_bn = tf.layers.batch_normalization(conv3_pool)
    
    #Layers 7,8:
    conv4 = tf.nn.conv2d(conv3_bn, conv4_filter, strides=[1,1,1,1], padding='SAME')
    conv4 = tf.nn.relu(conv4)
    conv4_pool = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv4_bn = tf.layers.batch_normalization(conv4_pool)
    
    #Layer 9:
    flat = tf.contrib.layers.flatten(conv4_bn)
    
    #Layer 10:
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    #Layer 11:
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    #Layer 12:
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=512, activation_fn=tf.nn.relu)
    full3 = tf.nn.dropout(full3, keep_prob)
    full3 = tf.layers.batch_normalization(full3)
    
    #Layer 13:
    full4 = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=1024, activation_fn=tf.nn.relu)
    full4 = tf.nn.dropout(full4, keep_prob)
    full4 = tf.layers.batch_normalization(full4)
    
    #Layer 14:
    out = tf.contrib.layers.fully_connected(inputs=full4, num_outputs=10, activation_fn=None)
    
    return out

In [27]:
epochs = 10
batch_size = 128
keep_prob = 0.7
learning_rate = 0.001

In [28]:
logits = convnet(x, keep_prob)
model = tf.identity(logits, name='logits')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

